In [1]:
%load_ext autoreload
%autoreload 2

import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense, Flatten, Input, Reshape
from keras.models import Sequential, Model
from PIL import Image
from pprint import pprint

from keras_preprocessing.image import DataFrameIterator, ImageDataGenerator


pixel_val = 1
np_filenames, img_filenames, np_arrays = [], [], []
for i in range(8):
    np_filename = '/tmp/{}.npy'.format(i)
    img_filename = '/tmp/{}.jpg'.format(i)
    array = pixel_val * np.random.randint(0, 255, size=(2, 2)).astype('float32')
    np.save(np_filename, array[..., None])
    plt.imsave(img_filename, array)
    
    np_filenames.append(np_filename)
    img_filenames.append(img_filename)
    
df = pd.DataFrame({'img_path': img_filenames, 'img_np_path': np_filenames})
df['regression'] = np.random.randint(1, 10, size=len(df))
df['binary'] = random.sample(['dog', 'cat'] * 30, len(df))
df['multi_class'] = random.sample(['dog', 'cat', 'horse'] * 30, len(df))
df['multi_label'] = random.sample(['dog', 'cat', ['dog'], ['cat'], ['cat', 'dog'], ['dog', 'cat']] * 10, len(df))
df['img_path_extra'] = img_filenames
df['regression_extra'] = np.random.randint(10, 100, size=len(df))

df.head()

Using TensorFlow backend.


,img_path,img_np_path,regression,binary,multi_class,multi_label,img_path_extra,regression_extra
0,/tmp/0.jpg,/tmp/0.npy,3,cat,cat,cat,/tmp/0.jpg,80
1,/tmp/1.jpg,/tmp/1.npy,7,cat,cat,[cat],/tmp/1.jpg,86
2,/tmp/2.jpg,/tmp/2.npy,1,cat,dog,dog,/tmp/2.jpg,79
3,/tmp/3.jpg,/tmp/3.npy,4,cat,horse,[cat],/tmp/3.jpg,36
4,/tmp/4.jpg,/tmp/4.npy,7,dog,cat,[dog],/tmp/4.jpg,44


### BInary classification

In [2]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['binary'],
    output_modes={'binary': 'sparse'},
    image_size=(2, 2),
    batch_size=2
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

print()
model = Sequential()
model.add(Flatten(input_shape=(2, 2, 3)))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

W1009 08:37:26.290768 140473523877696 deprecation.py:323] From /home/rodrigo/anaconda3/envs/keras-dev/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg']}]

Outputs:
[{'column': 'binary',
  'index_to_class': {0: 'cat', 1: 'dog'},
  'mode': 'sparse',
  'values': [0, 0, 0, 0, 1, 0, 0, 1]}]



W1009 08:37:26.511084 140473523877696 deprecation_wrapper.py:119] From /home/rodrigo/anaconda3/envs/keras-dev/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/1
4/4 [==============================] - 0s 26ms/step - loss: 52.4117


In [3]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_np_path',
    output_columns=['binary'],
    output_modes={'binary': 'sparse'},
    image_size=(2, 2),
    batch_size=2
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

print()
model = Sequential()
model.add(Flatten(input_shape=(2,2,3)))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_np_path',
  'values': ['/tmp/0.npy',
             '/tmp/1.npy',
             '/tmp/2.npy',
             '/tmp/3.npy',
             '/tmp/4.npy',
             '/tmp/5.npy',
             '/tmp/6.npy',
             '/tmp/7.npy']}]

Outputs:
[{'column': 'binary',
  'index_to_class': {0: 'cat', 1: 'dog'},
  'mode': 'sparse',
  'values': [0, 0, 0, 0, 1, 0, 0, 1]}]

Epoch 1/1
4/4 [==============================] - 0s 19ms/step - loss: 122.0674


### Multi-class classsification

In [4]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'sparse'},
    image_size=(2, 2),
    batch_size=2
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

print()
model = Sequential()
model.add(Flatten(input_shape=(2,2,3)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'sparse_categorical_crossentropy')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg']}]

Outputs:
[{'column': 'multi_class',
  'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'},
  'mode': 'sparse',
  'values': [0, 0, 1, 2, 0, 0, 1, 0]}]

Epoch 1/1
4/4 [==============================] - 0s 14ms/step - loss: 114.5474


In [5]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_np_path',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'sparse'},
    image_size=(2, 2),
    batch_size=2
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

print()
model = Sequential()
model.add(Flatten(input_shape=(2,2,3)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'sparse_categorical_crossentropy')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_np_path',
  'values': ['/tmp/0.npy',
             '/tmp/1.npy',
             '/tmp/2.npy',
             '/tmp/3.npy',
             '/tmp/4.npy',
             '/tmp/5.npy',
             '/tmp/6.npy',
             '/tmp/7.npy']}]

Outputs:
[{'column': 'multi_class',
  'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'},
  'mode': 'sparse',
  'values': [0, 0, 1, 2, 0, 0, 1, 0]}]

Epoch 1/1
4/4 [==============================] - 0s 17ms/step - loss: 246.6043


In [6]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'categorical'},
    image_size=(2, 2),
    batch_size=2,
    image_data_generator=ImageDataGenerator(horizontal_flip=True,
                                            vertical_flip=True)
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

print()
model = Sequential()
model.add(Flatten(input_shape=(2,2,3)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'categorical_crossentropy')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg']}]

Outputs:
[{'column': 'multi_class',
  'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'},
  'mode': 'categorical',
  'values': [0, 0, 1, 2, 0, 0, 1, 0]}]

Epoch 1/1
4/4 [==============================] - 0s 19ms/step - loss: 155.6045


In [7]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_np_path',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'categorical'},
    image_size=(2, 2),
    batch_size=2,
    image_data_generator=ImageDataGenerator(horizontal_flip=True,
                                            vertical_flip=True)
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

print()
model = Sequential()
model.add(Flatten(input_shape=(2,2,3)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'categorical_crossentropy')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_np_path',
  'values': ['/tmp/0.npy',
             '/tmp/1.npy',
             '/tmp/2.npy',
             '/tmp/3.npy',
             '/tmp/4.npy',
             '/tmp/5.npy',
             '/tmp/6.npy',
             '/tmp/7.npy']}]

Outputs:
[{'column': 'multi_class',
  'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'},
  'mode': 'categorical',
  'values': [0, 0, 1, 2, 0, 0, 1, 0]}]

Epoch 1/1
4/4 [==============================] - 0s 20ms/step - loss: 130.0609


### Multi-label classification

In [8]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_label'],
    output_modes={'multi_label': 'categorical'},
    image_size=(2, 2),
    batch_size=2,
    image_data_generator=ImageDataGenerator(horizontal_flip=True,
                                            vertical_flip=True)
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

print()
model = Sequential()
model.add(Flatten(input_shape=(2,2,3)))
model.add(Dense(2, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_path',
  'values': ['/tmp/0.jpg',
             '/tmp/1.jpg',
             '/tmp/2.jpg',
             '/tmp/3.jpg',
             '/tmp/4.jpg',
             '/tmp/5.jpg',
             '/tmp/6.jpg',
             '/tmp/7.jpg']}]

Outputs:
[{'column': 'multi_label',
  'index_to_class': {0: 'cat', 1: 'dog'},
  'mode': 'categorical',
  'values': [0, [0], 1, [0], [1], [1], [0, 1], [1, 0]]}]

Epoch 1/1
4/4 [==============================] - 0s 27ms/step - loss: 68.9422


In [9]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_np_path',
    output_columns=['multi_label'],
    output_modes={'multi_label': 'categorical'},
    image_size=(2, 2),
    batch_size=2,
    image_data_generator=ImageDataGenerator(horizontal_flip=True,
                                            vertical_flip=True)
)

print('\nInputs:')
pprint(df_iter.inputs)
print('\nOutputs:')
pprint(df_iter.outputs)

print()
model = Sequential()
model.add(Flatten(input_shape=(2,2,3)))
model.add(Dense(2, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)


Inputs:
[{'column': 'img_np_path',
  'values': ['/tmp/0.npy',
             '/tmp/1.npy',
             '/tmp/2.npy',
             '/tmp/3.npy',
             '/tmp/4.npy',
             '/tmp/5.npy',
             '/tmp/6.npy',
             '/tmp/7.npy']}]

Outputs:
[{'column': 'multi_label',
  'index_to_class': {0: 'cat', 1: 'dog'},
  'mode': 'categorical',
  'values': [0, [0], 1, [0], [1], [1], [0, 1], [1, 0]]}]

Epoch 1/1
4/4 [==============================] - 0s 45ms/step - loss: 66.2203
